In [1]:
# !pip install nbconvert 
# !jupyter nbconvert model_team14.ipynb --to script
#!pip install numpy -U

In [2]:
import model_team14 
from model_team14 import select_features, plot_pca, DTW

import os
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split, KFold, TimeSeriesSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [3]:
## 
metadata=pd.read_csv('../data/full_info.csv')
threshold=0.2
criteria=None  

path='../result/ft_{}_{}'.format(str(criteria), str(threshold))

if not os.path.exists(path):
    os.mkdir(path)

## filtering criteria is not cumulative explained variance ratio but just explained variance rati

In [4]:
#plot_pca(df_feature, metadata)

#df_feature=select_features(metadata, X, threshold, criteria=criteria)

In [5]:
def knndtw_result(metadata, threshold, criteria ,param_knn, y_type='y_agg' ,cv='block', test_year=3, n_splits=3, dtype='tr'): 

    if dtype=='tr':
        X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
    else:
        X=pd.read_csv('../data/X_data.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data.csv', index_col='date', parse_dates=True)
        
    df_feature=select_features(metadata, X, threshold, criteria=criteria)
    selected_features=list(df_feature[df_feature.select==1]['variable'])
    
    ## train and validation set: X_train, y_train / final test set: X_test, y_test
    X_train=X[selected_features][:-(test_year*12)]  
    y_train=y[y_type][:-(test_year*12)]           
    X_test=X[selected_features][-(test_year*12):]   
    y_test=y[y_type][-(test_year*12):]
    
    ## cross validation for parameter tuning & training
    if cv=='block':
        split=KFold(n_splits=n_splits, shuffle=False)  
    else:
        split=TimeSeriesSplit(n_splits=n_splits)
        
    clf = GridSearchCV(KNeighborsClassifier(metric=DTW), param_knn, cv=split, 
                       verbose=3, n_jobs=-1)
    
    clf.fit(X_train, y_train)
    
    ## training result
    y_pred_prob=clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)

    clf_report=classification_report(y_test, y_pred)

    param=clf.cv_results_['params']
    mean_test_score=clf.cv_results_['mean_test_score']
    rank_test_score=clf.cv_results_['rank_test_score']

    for idx, x in enumerate(param):
        x['model']='KNN'
        x['data']=dtype
        x['y']=y_type
        x['cv']=cv
        x['mean_test_score']=mean_test_score[idx] 
        x['rank_test_score']=rank_test_score[idx]

    df_cvresult=pd.DataFrame(param)

    with open (path+'/clf_knn_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'wb') as f:
        pickle.dump([clf, df_cvresult, y_pred, y_pred_prob, clf_report], f)


In [6]:
#df_feature=df_feature
param_knns=[{'n_neighbors':[2,3,4,5],
            'weights':['uniform','distance']}]  ## you can add more dictionary for other combinations of parameters.
y_types=['y_agg','y_oecd']
cvs=['block','tss']
test_year=5
n_splits=3
dtypes=['tr','ntr']

for param_knn in param_knns:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
                knndtw_result(metadata, threshold, criteria, param_knn, y_type=y_type ,cv=cv, test_year=test_year, n_splits=n_splits, dtype=dtype)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits


C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits


C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits


C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits


C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
dict_knn={}

for param_knn in param_knns:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
                with open (path+'/clf_knn_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'rb') as f:
                    [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
                    dict_knn['clf_knn_{}_{}_{}_t{}_spl{}'.format(y_type, dtype, cv, test_year, n_splits)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]
                    
for idx, model in enumerate(dict_knn):
    if idx==0:
        df_knn=dict_knn[model][1]
    else:
        df_tmp=dict_knn[model][1]
        df_knn=pd.concat([df_knn, df_tmp])
                    
df_knn[df_knn.rank_test_score==1]     
    

,n_neighbors,weights,model,data,y,cv,mean_test_score,rank_test_score
4,4,uniform,KNN,tr,y_agg,block,0.606061,1
0,2,uniform,KNN,ntr,y_agg,block,0.297348,1
1,2,distance,KNN,ntr,y_agg,block,0.297348,1
7,5,distance,KNN,tr,y_agg,tss,0.606061,1
0,2,uniform,KNN,ntr,y_agg,tss,0.388889,1
6,5,uniform,KNN,tr,y_oecd,block,0.657197,1
0,2,uniform,KNN,ntr,y_oecd,block,0.507576,1
1,2,distance,KNN,ntr,y_oecd,block,0.507576,1
6,5,uniform,KNN,tr,y_oecd,tss,0.661616,1
0,2,uniform,KNN,ntr,y_oecd,tss,0.434343,1


In [8]:
report=dict_knn['clf_knn_y_agg_ntr_block_t5_spl3'][-1]
lines = report.split('\n')
line=lines[-4]
row_data=[val for val in line.split(' ') if val!='']
accuracy=row_data[-2]

line=lines[-3]
row_data=[val for val in line.split(' ') if val!='']
macro_f1=row_data[-2]

line=lines[-2]
row_data=[val for val in line.split(' ') if val!='']
weighted_f1=row_data[-2]

print(accuracy, macro_f1, weighted_f1)
print(report)

0.93 0.32 0.90
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.97        56
         1.0       0.00      0.00      0.00         2
         2.0       0.00      0.00      0.00         2

    accuracy                           0.93        60
   macro avg       0.31      0.33      0.32        60
weighted avg       0.87      0.93      0.90        60



In [9]:
def classification_report_csv(report):
    report_data = []
    lines = report.split('\n')
    
    for line in lines[2:len(lines)-5]:
        row = {}
        row_data = [val for val in line.split(' ') if val!='']
        row['class'] = round(float(row_data[0]),0)
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        row['accuracy']=float([val for val in lines[-4].split(' ') if val!=''][-2])
        report_data.append(row)
        
    df = pd.DataFrame(report_data)
    return df

In [10]:
for idx, model in enumerate(dict_knn.keys()):
    
    report=dict_knn[model][-1]
    
    if idx==0:
        df_knn_creport=classification_report_csv(report)
        df_knn_creport['model']=model
    else:
        df_tmp=classification_report_csv(report)
        df_tmp['model']=model
        df_knn_creport=pd.concat([df_knn_creport, df_tmp])
        
df_knn_creport[df_knn_creport['class']>0]

,class,precision,recall,f1_score,support,accuracy,model
1,1.0,0.10,0.50,0.17,2.0,0.80,clf_knn_y_agg_tr_block_t5_spl3
2,2.0,0.33,0.50,0.40,2.0,0.80,clf_knn_y_agg_tr_block_t5_spl3
1,1.0,0.00,0.00,0.00,2.0,0.93,clf_knn_y_agg_ntr_block_t5_spl3
2,2.0,0.00,0.00,0.00,2.0,0.93,clf_knn_y_agg_ntr_block_t5_spl3
1,1.0,0.07,0.50,0.12,2.0,0.75,clf_knn_y_agg_tr_tss_t5_spl3
2,2.0,0.50,0.50,0.50,2.0,0.75,clf_knn_y_agg_tr_tss_t5_spl3
1,1.0,0.00,0.00,0.00,2.0,0.93,clf_knn_y_agg_ntr_tss_t5_spl3
2,2.0,0.00,0.00,0.00,2.0,0.93,clf_knn_y_agg_ntr_tss_t5_spl3
1,1.0,0.18,0.75,0.29,4.0,0.75,clf_knn_y_oecd_tr_block_t5_spl3
1,1.0,0.00,0.00,0.00,4.0,0.93,clf_knn_y_oecd_ntr_block_t5_spl3


In [11]:
def randomforest_result(metadata, threshold, criteria, param_rf, y_type='y_agg' ,cv='block', test_year=3, n_splits=3, dtype='tr'):
    
    if dtype=='tr':
        X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
    else:
        X=pd.read_csv('../data/X_data.csv', index_col='date', parse_dates=True)
        y=pd.read_csv('../data/y_data.csv', index_col='date', parse_dates=True)
    
    df_feature=select_features(metadata, X, threshold, criteria=criteria)
    selected_features=list(df_feature[df_feature.select==1]['variable'])
    
    ## train and validation set: X_train, y_train / final test set: X_test, y_test
    X_train=X[selected_features][:-(test_year*12)]  
    y_train=y[y_type][:-(test_year*12)]           
    X_test=X[selected_features][-(test_year*12):]   
    y_test=y[y_type][-(test_year*12):]
    
    ## cross validation for parameter tuning & training
    if cv=='block':
        split=KFold(n_splits=n_splits, shuffle=False)  
    else:
        split=TimeSeriesSplit(n_splits=n_splits)
        
    clf = GridSearchCV(RandomForestClassifier(n_jobs=-1, oob_score=True), 
                       param_rf, cv=split, 
                       verbose=3, n_jobs=-1)
    
    clf.fit(X_train, y_train)
    
    ## training result
    y_pred_prob=clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)

    clf_report=classification_report(y_test, y_pred)

    param=clf.cv_results_['params']
    mean_test_score=clf.cv_results_['mean_test_score']
    rank_test_score=clf.cv_results_['rank_test_score']

    for idx, x in enumerate(param):
        x['model']='RF'
        x['data']=dtype
        x['y']=y_type
        x['cv']=cv
        x['mean_test_score']=mean_test_score[idx] 
        x['rank_test_score']=rank_test_score[idx]

    df_cvresult=pd.DataFrame(param)

    with open (path+'/clf_rf_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'wb') as f:
        pickle.dump([clf, df_cvresult, y_pred, y_pred_prob, clf_report], f)


In [12]:
#df_feature=df_feature
param_rfs=[{'n_estimators':[10,30,50,70,100],
             'max_features':[0.1, 0.2, 0.3, 0.5, 0.7, 0.9]}]  ## you can add more dictionary for other combinations of parameters.
y_types=['y_agg','y_oecd']
cvs=['block','tss']
test_year=5
n_splits=3
dtypes=['tr','ntr']

for param_rf in param_rfs:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
                randomforest_result(metadata, threshold, criteria, param_rf, y_type=y_type ,cv=cv, test_year=test_year, n_splits=n_splits, dtype=dtype)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\gredi\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gredi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and bein

Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\gredi\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\gredi\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\gredi\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


In [13]:
dict_rf={}

for param_rf in param_rfs:
    for y_type in y_types:
        for cv in cvs:
            for dtype in dtypes:
                with open (path+'/clf_rf_{}_{}_{}_t{}_spl{}.pkl'.format(y_type, dtype, cv, test_year, n_splits), 'rb') as f:
                    [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)
                    dict_rf['clf_rf_{}_{}_{}_t{}_spl{}'.format(y_type, dtype, cv, test_year, n_splits)]=[clf, df_cvresult, y_pred, y_pred_prob, clf_report]
                    
for idx, model in enumerate(dict_rf):
    if idx==0:
        df_rf=dict_rf[model][1]
    else:
        df_tmp=dict_rf[model][1]
        df_rf=pd.concat([df_rf, df_tmp])
                    
df_rf[df_rf.rank_test_score==1]    

,max_features,n_estimators,model,data,y,cv,mean_test_score,rank_test_score
3,0.1,70,RF,tr,y_agg,block,0.642045,1
6,0.2,30,RF,ntr,y_agg,block,0.545455,1
3,0.1,70,RF,tr,y_agg,tss,0.611111,1
4,0.1,100,RF,tr,y_agg,tss,0.611111,1
20,0.7,10,RF,tr,y_agg,tss,0.611111,1
15,0.5,10,RF,ntr,y_agg,tss,0.494949,1
5,0.2,10,RF,tr,y_oecd,block,0.678030,1
0,0.1,10,RF,ntr,y_oecd,block,0.553030,1
8,0.2,70,RF,tr,y_oecd,tss,0.671717,1
9,0.2,100,RF,tr,y_oecd,tss,0.671717,1


In [14]:
for idx, model in enumerate(dict_rf.keys()):
    
    report=dict_rf[model][-1]
    
    if idx==0:
        df_rf_creport=classification_report_csv(report)
        df_rf_creport['model']=model
    else:
        df_tmp=classification_report_csv(report)
        df_tmp['model']=model
        df_rf_creport=pd.concat([df_rf_creport, df_tmp])
        
df_rf_creport


####
#'clf_rf_y_agg_tr_block_t5_spl3' model looks good
# according to df_rf, it seems that high n_estimators and low max_features work well!
# we can try it for other parameter combinations

,class,precision,recall,f1_score,support,accuracy,model
0,0.0,0.98,0.82,0.89,56.0,0.82,clf_rf_y_agg_tr_block_t5_spl3
1,1.0,0.10,0.50,0.17,2.0,0.82,clf_rf_y_agg_tr_block_t5_spl3
2,2.0,0.67,1.00,0.80,2.0,0.82,clf_rf_y_agg_tr_block_t5_spl3
0,0.0,0.96,0.77,0.85,56.0,0.73,clf_rf_y_agg_ntr_block_t5_spl3
1,1.0,0.07,0.50,0.12,2.0,0.73,clf_rf_y_agg_ntr_block_t5_spl3
2,2.0,0.00,0.00,0.00,2.0,0.73,clf_rf_y_agg_ntr_block_t5_spl3
0,0.0,0.96,0.84,0.90,56.0,0.82,clf_rf_y_agg_tr_tss_t5_spl3
1,1.0,0.00,0.00,0.00,2.0,0.82,clf_rf_y_agg_tr_tss_t5_spl3
2,2.0,0.67,1.00,0.80,2.0,0.82,clf_rf_y_agg_tr_tss_t5_spl3
0,0.0,0.98,0.73,0.84,56.0,0.70,clf_rf_y_agg_ntr_tss_t5_spl3


In [15]:
dict_list=[df_knn, df_knn_creport, df_rf, df_rf_creport]
id_list=['df_knn', 'df_knn_creport', 'df_rf', 'df_rf_creport']

for i, dict_ in enumerate(dict_list):
    with open(path+'/{}.pkl'.format(id_list[i]), 'wb') as f:
        pickle.dump(dict_, f)